In [1]:
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df

[2020-11-21 10:31:47,470] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-21 10:31:47,498] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-21 10:31:47,524] - views.utils.io:65 - DEBUG - Reading parquet at /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2020-11-21 10:31:47,991] - views.utils.io:72 - DEBUG - Finished reading parquet from /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [2]:
run_id = "d_2020_04_01"
periods = get_periods(run_id) # as a list
periods_by_name = get_periods_by_name(run_id)# as a dict

period_a = periods_by_name["A"]

period_b = periods_by_name["B"]
period_c = periods_by_name["C"]
period_a

[2020-11-21 10:31:50,004] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-21 10:31:50,040] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml


Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432)

In [4]:

model_from_pipeline_spec = all_cm_models_by_name["cm_sb_acled_violence"]
model_from_pipeline_spec1 = all_cm_models_by_name["cm_sb_cflong"]
model_from_pipeline_spec2 = all_cm_models_by_name["cm_sb_neibhist"]
model_from_pipeline_spec3 = all_cm_models_by_name["cm_sb_cdummies"]
model_from_pipeline_spec4 = all_cm_models_by_name["cm_sb_acled_protest"]
model_from_pipeline_spec5 = all_cm_models_by_name["cm_sb_reign_coups"]
model_from_pipeline_spec6 = all_cm_models_by_name["cm_sb_icgcw"]
model_from_pipeline_spec7 = all_cm_models_by_name["cm_sb_reign_drought"]
model_from_pipeline_spec8 = all_cm_models_by_name["cm_sb_reign_global"]
model_from_pipeline_spec9 = all_cm_models_by_name["cm_sb_vdem_global"]
model_from_pipeline_spec10 = all_cm_models_by_name["cm_sb_demog"]
model_from_pipeline_spec11= all_cm_models_by_name["cm_sb_wdi_global"]
model_from_pipeline_spec12 = all_cm_models_by_name["cm_sb_all_global"]
model_from_pipeline_spec13 = all_cm_models_by_name["cm_sbonset24_25_all"]



models=[
        model_from_pipeline_spec ,
        model_from_pipeline_spec1,
        model_from_pipeline_spec2,
        model_from_pipeline_spec3 ,
        model_from_pipeline_spec4,
        model_from_pipeline_spec5,
        model_from_pipeline_spec6 ,
        model_from_pipeline_spec7,
        model_from_pipeline_spec8,
        model_from_pipeline_spec9 ,
        model_from_pipeline_spec10,
        model_from_pipeline_spec11,
        model_from_pipeline_spec12,
        model_from_pipeline_spec13
       ]
for model in models:
    model.periods = periods

In [5]:
cflong_acled_violence_ensemble = Ensemble(
    name="cflong_acled_violence_ensemble", 
    models=models, 
    outcome_type="prob", 
    col_outcome="greq_25_ged_best_sb", 
    method="average", 
    periods=periods
)
ensembles = [cflong_acled_violence_ensemble]


In [6]:
for model in models:
    model.periods = periods

In [7]:
for model in models:
    
    # Uncalibrated predictions
    df_pred = model.predict(df)
    # assign_into_df takes care to only overwrite rows with actual values
    # This way we can keep all periods in the same df
    # It's also idempotent, no joining, so run as many times as you like. 
    df = assign_into_df(df_to=df, df_from=df_pred)
    
#     df_pred = model.predict_calibrated(
#         df=df.fillna(0), 
#         period_calib=period_a,
#         period_test=period_b,)
        
#     df = assign_into_df(df_to=df, df_from=df_pred)
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)
    

[2020-11-21 10:42:34,961] - views.apps.model.api:552 - INFO - Predicting for cm_sb_acled_violence
[2020-11-21 10:42:34,962] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432), Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468), Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)]
[2020-11-21 10:42:34,996] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_1.joblib
[2020-11-21 10:42:41,532] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_3.joblib
[2020-11-21 10:42:47,767] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_6.joblib
[2020-11-21 10:42:54,321] - views.apps.model.api:125 - DEBUG - Loading /home/ky

[2020-11-21 10:47:19,057] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)]
[2020-11-21 10:47:19,077] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_1.joblib
[2020-11-21 10:47:27,394] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_3.joblib
[2020-11-21 10:47:35,558] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_6.joblib
[2020-11-21 10:47:43,911] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_9.joblib
[2020-11-21 10:47:52,272] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_12.joblib
[2020-11-21 10:48:00,530] - views.apps.model

[2020-11-21 10:52:07,963] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_C_30.joblib
[2020-11-21 10:52:16,048] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_C_36.joblib
[2020-11-21 10:52:24,872] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_C_38.joblib
[2020-11-21 10:52:33,307] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_1
[2020-11-21 10:52:33,508] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_3
[2020-11-21 10:52:33,711] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_6
[2020-11-21 10:52:33,919] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_9
[2020-11-21 10:52:34,124] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_12
[2020-11-21 10:52:34,326] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_cflong_18
[2020-11-21 10:52

[2020-11-21 10:55:41,288] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_A_18.joblib
[2020-11-21 10:55:47,394] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_A_24.joblib
[2020-11-21 10:55:53,344] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_A_30.joblib
[2020-11-21 10:56:00,476] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_A_36.joblib
[2020-11-21 10:56:07,091] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_A_38.joblib
[2020-11-21 10:56:13,243] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_neibhist_B_1.joblib
[2020-11-21 10:56:20,308] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models

[2020-11-21 11:01:05,289] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 1
[2020-11-21 11:01:05,299] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 3
[2020-11-21 11:01:05,308] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 6
[2020-11-21 11:01:05,316] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 9
[2020-11-21 11:01:05,327] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 12
[2020-11-21 11:01:05,338] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 18
[2020-11-21 11:01:05,348] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 24
[2020-11-21 11:01:05,358] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 30
[2020-11-21 11:01:05,368] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 36
[2020-11-21 11:01:05,379] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_neibhist step 38
[2020-11-21 11:01:05,424

...

[2020-11-21 11:45:29,921] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_6.joblib
[2020-11-21 11:45:34,955] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_9.joblib
[2020-11-21 11:45:40,325] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_12.joblib
[2020-11-21 11:45:45,275] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_18.joblib
[2020-11-21 11:45:50,286] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_24.joblib
[2020-11-21 11:45:55,312] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_wdi_global_B_30.joblib
[2020-11-21 11:46:00,330] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/sto

[2020-11-21 11:48:51,181] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_wdi_global step 36
[2020-11-21 11:48:51,191] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_wdi_global step 38
[2020-11-21 11:48:51,233] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_1_calibrated
[2020-11-21 11:48:51,440] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_3_calibrated
[2020-11-21 11:48:51,651] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_6_calibrated
[2020-11-21 11:48:51,865] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_9_calibrated
[2020-11-21 11:48:52,076] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_12_calibrated
[2020-11-21 11:48:52,285] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_18_calibrated
[2020-11-21 11:48:52,497] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_wdi_global_24_calibrated
[2020-11-21 11:48:52,705] - views.utils.data:110 - DEBU

[2020-11-21 11:52:03,571] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_9.joblib
[2020-11-21 11:52:09,046] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_12.joblib
[2020-11-21 11:52:14,914] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_18.joblib
[2020-11-21 11:52:20,422] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_24.joblib
[2020-11-21 11:52:25,785] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_30.joblib
[2020-11-21 11:52:31,165] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_B_36.joblib
[2020-11-21 11:52:36,371] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/st

[2020-11-21 11:55:01,858] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_B_38.joblib
[2020-11-21 11:55:05,792] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_1.joblib
[2020-11-21 11:55:09,682] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_3.joblib
[2020-11-21 11:55:13,626] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_6.joblib
[2020-11-21 11:55:17,818] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_9.joblib
[2020-11-21 11:55:21,969] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_12.joblib
[2020-11-21 11:55:25,770] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views

[2020-11-21 11:57:11,777] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_1_calibrated
[2020-11-21 11:57:12,006] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_3_calibrated
[2020-11-21 11:57:12,308] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_6_calibrated
[2020-11-21 11:57:12,558] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_9_calibrated
[2020-11-21 11:57:12,784] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_12_calibrated
[2020-11-21 11:57:13,030] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_18_calibrated
[2020-11-21 11:57:13,267] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_24_calibrated
[2020-11-21 11:57:13,492] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_30_calibrated
[2020-11-21 11:57:13,721] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sbonset24_25_all_36_calibrated
[2020-11-21 11

In [11]:
for ensemble in ensembles:
    df_pred = ensemble.predict(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)

[2020-11-21 11:59:16,723] - views.apps.model.api:814 - INFO - Predicting for cflong_acled_violence_ensemble
[2020-11-21 11:59:16,995] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_1
[2020-11-21 11:59:17,006] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_3
[2020-11-21 11:59:17,021] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_6
[2020-11-21 11:59:17,036] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_9
[2020-11-21 11:59:17,050] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_12
[2020-11-21 11:59:17,062] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_18
[2020-11-21 11:59:17,072] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_24
[2020-11-21 11:59:17,086] - views.utils.data:110 - DEBUG - Inserting col ss_cflong_acled_violence_ensemble_30
[2020-11-21 11:5

In [9]:
df.head(4)

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        40                     0.0             0.0             0.0   
         41                     0.0             0.0             0.0   
         42                     0.0             0.0             0.0   
         43                     0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        40                     0.0               0               0   
         41                     0.0               0               0   
         42                     0.0               0               0   
         43                     0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        40                       0               0       0        0  ...   
         41                       0               0       0        0  ...   
         42                       0               0       0        0  ...   
         43                       0               0       0        0  ...   

                     ss_cflong_acled_violence_ensemble_3  \
month_id country_id                                        
1        40                                          NaN   
         41                                          NaN   
         42                                          NaN   
         43                                          NaN   

                     ss_cflong_acled_violence_ensemble_6  \
month_id country_id                                        
1        40                                          NaN   
         41                                          NaN   
         42                                          NaN   
         43                                          NaN   

                     ss_cflong_acled_violence_ensemble_9  \
month_id country_id                                        
1        40                                          NaN   
         41                                          NaN   
         42                                          NaN   
         43                                          NaN   

                     ss_cflong_acled_violence_ensemble_12  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cflong_acled_violence_ensemble_18  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cflong_acled_violence_ensemble_24  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cflong_acled_violence_ensemble_30  \
month_id country_id                                         
1        40                                           NaN   
         41                                           NaN   
         42                                           NaN   
         43                                           NaN   

                     ss_cflong_acled_violence_ensemble_36  \
month_id country_id                                         
1        40                    

In [12]:
type(df)
df=df.fillna(0)
df.head(4)

acled_count_ns  acled_count_os  acled_count_pr  \
month_id country_id                                                   
1        40                     0.0             0.0             0.0   
         41                     0.0             0.0             0.0   
         42                     0.0             0.0             0.0   
         43                     0.0             0.0             0.0   

                     acled_count_sb  acled_dummy_ns  acled_dummy_os  \
month_id country_id                                                   
1        40                     0.0               0               0   
         41                     0.0               0               0   
         42                     0.0               0               0   
         43                     0.0               0               0   

                     acled_dummy_pr  acled_dummy_sb  cdum_1  cdum_10  ...  \
month_id country_id                                                   ...   
1        40                       0               0       0        0  ...   
         41                       0               0       0        0  ...   
         42                       0               0       0        0  ...   
         43                       0               0       0        0  ...   

                     ss_cflong_acled_violence_ensemble_3  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_6  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_9  \
month_id country_id                                        
1        40                                          0.0   
         41                                          0.0   
         42                                          0.0   
         43                                          0.0   

                     ss_cflong_acled_violence_ensemble_12  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_18  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_24  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_30  \
month_id country_id                                         
1        40                                           0.0   
         41                                           0.0   
         42                                           0.0   
         43                                           0.0   

                     ss_cflong_acled_violence_ensemble_36  \
month_id country_id                                         
1        40                    

In [13]:
for ensemble in ensembles:
    ensemble.evaluate(df, period=periods_by_name["B"])
    ensemble.evaluate(df, period=periods_by_name["C"])

[2020-11-21 11:59:27,646] - views.apps.model.api:966 - INFO - Evaluating cflong_acled_violence_ensemble
[2020-11-21 11:59:27,650] - views.apps.model.api:970 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step-combined
[2020-11-21 11:59:27,808] - views.apps.model.api:992 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble period B step-combined
[2020-11-21 11:59:27,813] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step 1
[2020-11-21 11:59:27,866] - views.apps.model.api:1033 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble period B step 1
[2020-11-21 11:59:27,870] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cflong_acled_violence_ensemble period B step 3
[2020-11-21 11:59:27,921] - views.apps.model.api:1033 - DEBUG - Evaluating calibrated predictions for cflong_acled_violence_ensemble 

In [14]:
for ensemble in ensembles:
    print(ensemble.name)
    print("Weights:")
    print(json.dumps(ensemble.weights, indent=2))
    print("Eval scores:")
    print(json.dumps(ensemble.evaluation.scores, indent=2))
    print("#"*80)

cflong_acled_violence_ensemble
Weights:
{
  "B": {
    "1": {
      "cm_sb_acled_violence": 0.07142857142857142,
      "cm_sb_cflong": 0.07142857142857142,
      "cm_sb_neibhist": 0.07142857142857142,
      "cm_sb_cdummies": 0.07142857142857142,
      "cm_sb_acled_protest": 0.07142857142857142,
      "cm_sb_reign_coups": 0.07142857142857142,
      "cm_sb_icgcw": 0.07142857142857142,
      "cm_sb_reign_drought": 0.07142857142857142,
      "cm_sb_reign_global": 0.07142857142857142,
      "cm_sb_vdem_global": 0.07142857142857142,
      "cm_sb_demog": 0.07142857142857142,
      "cm_sb_wdi_global": 0.07142857142857142,
      "cm_sb_all_global": 0.07142857142857142,
      "cm_sbonset24_25_all": 0.07142857142857142
    },
    "3": {
      "cm_sb_acled_violence": 0.07142857142857142,
      "cm_sb_cflong": 0.07142857142857142,
      "cm_sb_neibhist": 0.07142857142857142,
      "cm_sb_cdummies": 0.07142857142857142,
      "cm_sb_acled_protest": 0.07142857142857142,
      "cm_sb_reign_coups": 0.0